In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv(r'all_matches.csv')

C:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
df= df[(df['ball']<6)&((df['innings']==1) | (df['innings']==2))]

In [5]:
df.drop(['penalty','other_wicket_type','other_player_dismissed'], axis=1, inplace = True)

In [6]:
df = df.fillna(0)

In [7]:
overs = pd.DataFrame(df['ball'].astype(str))

In [8]:
df['overs'] = overs['ball'].apply(lambda x: int(x.split('.')[0])+1)

In [9]:
df['score'] = df['runs_off_bat']+df['extras']

In [10]:
df['total_score']=df.groupby(['match_id','innings'])['score'].cumsum()

In [11]:
df['wicket'] = df['player_dismissed'].apply(lambda x: 1 if x!=0 else 0)

In [12]:
df['bowlers_wicket']=df[df['wicket_type']!='run out']['wicket']

In [13]:
df

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,noballs,byes,legbyes,wicket_type,player_dismissed,overs,score,total_score,wicket,bowlers_wicket
0,335982,2007/08,18-04-2008,M Chinnaswamy Stadium,1,0.1,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,BB McCullum,...,0.0,0.0,1.0,0,0,1,1,1,0,0.0
1,335982,2007/08,18-04-2008,M Chinnaswamy Stadium,1,0.2,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,...,0.0,0.0,0.0,0,0,1,0,1,0,0.0
2,335982,2007/08,18-04-2008,M Chinnaswamy Stadium,1,0.3,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,...,0.0,0.0,0.0,0,0,1,1,2,0,0.0
3,335982,2007/08,18-04-2008,M Chinnaswamy Stadium,1,0.4,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,...,0.0,0.0,0.0,0,0,1,0,2,0,0.0
4,335982,2007/08,18-04-2008,M Chinnaswamy Stadium,1,0.5,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,...,0.0,0.0,0.0,0,0,1,0,2,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194260,1254060,2021,11-04-2021,"MA Chidambaram Stadium, Chepauk, Chennai",2,5.2,Sunrisers Hyderabad,Kolkata Knight Riders,JM Bairstow,MK Pandey,...,0.0,0.0,0.0,0,0,6,0,33,0,0.0
194261,1254060,2021,11-04-2021,"MA Chidambaram Stadium, Chepauk, Chennai",2,5.3,Sunrisers Hyderabad,Kolkata Knight Riders,JM Bairstow,MK Pandey,...,0.0,0.0,0.0,0,0,6,1,34,0,0.0
194262,1254060,2021,11-04-2021,"MA Chidambaram Stadium, Chepauk, Chennai",2,5.4,Sunrisers Hyderabad,Kolkata Knight Riders,MK Pandey,JM Bairstow,...,0.0,0.0,0.0,0,0,6,0,34,0,0.0
194263,1254060,2021,11-04-2021,"MA Chidambaram Stadium, Chepauk, Chennai",2,5.5,Sunrisers Hyderabad,Kolkata Knight Riders,MK Pandey,JM Bairstow,...,0.0,0.0,0.0,0,0,6,0,34,0,0.0


In [212]:
batsman = pd.DataFrame(df['striker'].unique(),columns=['striker'])

In [213]:
runs_batsman = df.groupby('striker')['score'].sum()

In [214]:
batsman['runs']= batsman['striker'].apply(lambda x: runs_batsman[x])

In [215]:
balls_faced = df.groupby('striker')['ball'].count()

In [216]:
batsman['balls_faced']=batsman['striker'].apply(lambda x : balls_faced[x])

In [217]:
batsman['strike_rate']=(batsman['runs']/batsman['balls_faced'])*100

In [218]:
batsman['matches_played']=batsman['striker'].apply(lambda x: df[df['striker']==x]['match_id'].nunique())

In [219]:
dismissals = df.groupby('player_dismissed')['wicket'].sum()

In [220]:
batsman['dismissals'] = batsman['striker'].apply(lambda x: dismissals[x] if x in dismissals else 0)

In [221]:
batsman['avg_runs']=batsman['runs']/batsman['dismissals']

In [222]:
batsman = batsman.replace(np.inf,0)

In [223]:
batsman.columns

Index(['striker', 'runs', 'balls_faced', 'strike_rate', 'matches_played',
       'dismissals', 'avg_runs'],
      dtype='object')

In [224]:
non_striker_zero =[]
for i in df['non_striker'].unique():
    if(i not in df['striker'].unique()):
        non_striker_zero += [i]

In [225]:
non_striker_zero = pd.DataFrame(non_striker_zero,columns=['striker'])

In [226]:
non_striker_zero['runs'] = 0

In [227]:
non_striker_zero['balls_faced']=0

In [228]:
non_striker_zero['strike_rate']=0

In [229]:
non_striker_zero['matches_played']=non_striker_zero['striker'].apply(lambda x: df[df['non_striker']==x]['match_id'].nunique())

In [230]:
non_striker_zero['dismissals']=0

In [231]:
non_striker_zero['avg_runs']=0

In [232]:
batsman = batsman.append(non_striker_zero)

In [233]:
batsman.reset_index(inplace=True)

In [234]:
batsman

,index,striker,runs,balls_faced,strike_rate,matches_played,dismissals,avg_runs
0,0,SC Ganguly,571,540,105.740741,46,19,30.052632
1,1,BB McCullum,1688,1273,132.600157,104,57,29.614035
2,2,RT Ponting,47,75,62.666667,9,5,9.400000
3,3,R Dravid,970,855,113.450292,64,17,57.058824
4,4,W Jaffer,83,95,87.368421,8,6,13.833333
...,...,...,...,...,...,...,...,...
284,284,Washington Sundar,16,17,94.117647,1,1,16.000000
285,285,RM Patidar,9,9,100.000000,1,1,9.000000
286,0,TR Birt,0,0,0.000000,1,0,0.000000
287,1,MR Marsh,0,0,0.000000,1,0,0.000000


In [235]:
batsman['striker']=batsman['striker'].astype(str)

In [238]:
batsman['striker'] = batsman['striker'].str.rstrip()

In [239]:
new_player = pd.read_csv(r'playersList.csv')

In [118]:
a = new_player['Players']

In [119]:
b = batsman['striker']

In [253]:
a =0

In [258]:
new_ = new_player['Players'].apply(lambda x: True if x.strip() not in batsman['striker'] else False )

In [259]:
new_player[new_]

,Players
0,AM Rahane
1,RA Tripathi
2,MJ McClenaghan
3,SPD Smith
4,SL Malinga
...,...
578,SS Agarwal
579,NB Singh
580,MJ Henry
581,JL Denly


In [248]:
new_player.iloc[285]==batsman['striker'].iloc[0]

Players    True
Name: 285, dtype: bool

In [249]:
batsman['striker'].iloc[0]

'SC Ganguly'

In [251]:
len(new_)

583

In [161]:
bowler=pd.DataFrame(df['bowler'].unique(),columns=['bowler'])

In [162]:
balls_bowled = df.groupby('bowler')['ball'].count()

In [163]:
bowler['balls_bowled'] = bowler['bowler'].apply(lambda x : balls_bowled[x])

In [164]:
wickets = df.groupby('bowler')['bowlers_wicket'].sum()

In [165]:
bowler['wickets']= bowler['bowler'].apply(lambda x: wickets[x])

In [166]:
bowler['strike_rate']=bowler['balls_bowled']/bowler['wickets']

In [167]:
bowler = bowler.replace(np.inf,0)

In [168]:
over = pd.DataFrame(df.groupby(['match_id','bowler'])['overs'].nunique())

In [169]:
over.reset_index(inplace=True)

In [170]:
bowler['overs'] = bowler['bowler'].apply(lambda x: over[over['bowler']==x]['overs'].sum())

In [171]:
bowler['runs_conceded'] = bowler['bowler'].apply(lambda x: df[df['bowler']==x]['score'].sum())

In [172]:
bowler['economy'] = bowler['runs_conceded']/bowler['overs']

In [173]:
bowler['bowling_avg'] = bowler['runs_conceded']/bowler['wickets']

In [174]:
bowler = bowler.replace(np.inf,0)

In [175]:
bowler.index = bowler['bowler']

In [176]:
bowler

,bowler,balls_bowled,wickets,strike_rate,overs,runs_conceded,economy,bowling_avg
bowler,,,,,,,,
P Kumar,P Kumar,1642,40.0,41.050000,262,1813,6.919847,45.325000
Z Khan,Z Khan,1346,53.0,25.396226,217,1492,6.875576,28.150943
AA Noffke,AA Noffke,7,0.0,0.000000,1,23,23.000000,0.000000
AB Dinda,AB Dinda,855,33.0,25.909091,135,1010,7.481481,30.606061
I Sharma,I Sharma,1267,43.0,29.465116,203,1493,7.354680,34.720930
...,...,...,...,...,...,...,...,...
CJ Green,CJ Green,12,0.0,0.000000,2,16,8.000000,0.000000
DR Sams,DR Sams,33,0.0,0.000000,5,38,7.600000,0.000000
Monu Kumar,Monu Kumar,8,0.0,0.000000,1,7,7.000000,0.000000


In [177]:
venue = pd.DataFrame(df.groupby('venue')['score'].sum())

In [178]:
venue['no_matches'] = df.groupby(['venue'])['match_id'].nunique()*2

In [179]:
venue['avg_score'] = venue['score']/venue['no_matches']

In [180]:
venue

,score,no_matches,avg_score
venue,,,
Arun Jaitley Stadium,1419,28,50.678571
Barabati Stadium,610,14,43.571429
Brabourne Stadium,1141,22,51.863636
Buffalo Park,237,6,39.500000
De Beers Diamond Oval,240,6,40.000000
Dr DY Patil Sports Academy,1439,34,42.323529
Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium,1065,26,40.961538
Dubai International Cricket Stadium,2951,66,44.712121
Eden Gardens,7125,154,46.266234


In [181]:
batting_team = pd.DataFrame(df.groupby('batting_team')['score'].sum())

In [182]:
batting_team.reset_index(inplace=True)

In [183]:
batting_team.index = batting_team['batting_team']

In [184]:
batting_team['no_matches'] = batting_team['batting_team'].apply(lambda x: df[df['batting_team']==x]['match_id'].nunique())

In [185]:
batting_team['avg_score'] = batting_team['score']/batting_team['no_matches']

In [186]:
batting_team

,batting_team,score,no_matches,avg_score
batting_team,,,,
Chennai Super Kings,Chennai Super Kings,8081,179,45.145251
Deccan Chargers,Deccan Chargers,3417,75,45.560000
Delhi Capitals,Delhi Capitals,1625,34,47.794118
Delhi Daredevils,Delhi Daredevils,7349,161,45.645963
Gujarat Lions,Gujarat Lions,1553,30,51.766667
Kings XI Punjab,Kings XI Punjab,8907,190,46.878947
Kochi Tuskers Kerala,Kochi Tuskers Kerala,680,14,48.571429
Kolkata Knight Riders,Kolkata Knight Riders,8921,193,46.222798
Mumbai Indians,Mumbai Indians,9245,204,45.318627


In [187]:
batsman[batsman['striker']=='TR Birt']

,index,striker,runs,balls_faced,strike_rate,matches_played,dismissals,avg_runs
striker,,,,,,,,
TR Birt,0,TR Birt,0,0,0.0,1,0,0.0


In [188]:
bowling_team = pd.DataFrame(df.groupby('bowling_team')['score'].sum())

In [189]:
bowling_team.reset_index(inplace=True)

In [190]:
bowling_team.index = bowling_team['bowling_team']

In [191]:
bowling_team['no_matches'] = bowling_team['bowling_team'].apply(lambda x: df[df['bowling_team']==x]['match_id'].nunique())

In [192]:
bowling_team['avg_score_conceded'] = bowling_team['score']/bowling_team['no_matches']

In [193]:
bowling_team

,bowling_team,score,no_matches,avg_score_conceded
bowling_team,,,,
Chennai Super Kings,Chennai Super Kings,8228,179,45.966480
Deccan Chargers,Deccan Chargers,3228,75,43.040000
Delhi Capitals,Delhi Capitals,1590,34,46.764706
Delhi Daredevils,Delhi Daredevils,7471,160,46.693750
Gujarat Lions,Gujarat Lions,1490,30,49.666667
Kings XI Punjab,Kings XI Punjab,9103,190,47.910526
Kochi Tuskers Kerala,Kochi Tuskers Kerala,593,14,42.357143
Kolkata Knight Riders,Kolkata Knight Riders,8897,193,46.098446
Mumbai Indians,Mumbai Indians,8914,204,43.696078


In [194]:
df1 = df[['venue','innings','batting_team','bowling_team','striker','non_striker','bowler','score']]

In [195]:
df1['striker'] = df1['striker'].replace(batsman['strike_rate'])

<ipython-input-195-b225eae5d776>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['striker'] = df1['striker'].replace(batsman['strike_rate'])


In [196]:
df1['non_striker'] = df1['non_striker'].replace(batsman['strike_rate'])

<ipython-input-196-53b111791819>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['non_striker'] = df1['non_striker'].replace(batsman['strike_rate'])


In [197]:
df1['bowler'] = df1['bowler'].replace(bowler['economy'])

<ipython-input-197-7dff4a34ada9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['bowler'] = df1['bowler'].replace(bowler['economy'])


In [198]:
df1['venue'] = df1['venue'].replace(venue['avg_score'])

<ipython-input-198-a56d82ade005>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['venue'] = df1['venue'].replace(venue['avg_score'])


In [199]:
batting_team['avg_score']

batting_team
Chennai Super Kings            45.145251
Deccan Chargers                45.560000
Delhi Capitals                 47.794118
Delhi Daredevils               45.645963
Gujarat Lions                  51.766667
Kings XI Punjab                46.878947
Kochi Tuskers Kerala           48.571429
Kolkata Knight Riders          46.222798
Mumbai Indians                 45.318627
Pune Warriors                  42.111111
Rajasthan Royals               44.693750
Rising Pune Supergiant         49.062500
Rising Pune Supergiants        45.571429
Royal Challengers Bangalore    44.617347
Sunrisers Hyderabad            47.616000
Name: avg_score, dtype: float64

In [200]:
df1['batting_team'] = df1['batting_team'].replace(batting_team['avg_score'])

<ipython-input-200-1ea7485e80e0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['batting_team'] = df1['batting_team'].replace(batting_team['avg_score'])


In [201]:
df1['bowling_team'] = df1['bowling_team'].replace(bowling_team['avg_score_conceded'])

<ipython-input-201-db31c42c8865>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['bowling_team'] = df1['bowling_team'].replace(bowling_team['avg_score_conceded'])


In [202]:

df1

,venue,innings,batting_team,bowling_team,striker,non_striker,bowler,score
0,44.653846,1,46.222798,46.256410,105.740741,132.600157,6.919847,1
1,44.653846,1,46.222798,46.256410,132.600157,105.740741,6.919847,0
2,44.653846,1,46.222798,46.256410,132.600157,105.740741,6.919847,1
3,44.653846,1,46.222798,46.256410,132.600157,105.740741,6.919847,0
4,44.653846,1,46.222798,46.256410,132.600157,105.740741,6.919847,0
...,...,...,...,...,...,...,...,...
194260,43.000000,2,47.616000,46.098446,134.000000,116.872891,7.964912,0
194261,43.000000,2,47.616000,46.098446,134.000000,116.872891,7.964912,1
194262,43.000000,2,47.616000,46.098446,116.872891,134.000000,7.964912,0
194263,43.000000,2,47.616000,46.098446,116.872891,134.000000,7.964912,0


In [203]:
X=df1.drop('score',axis=1).values
y=df1['score'].values

In [204]:
from sklearn.model_selection import train_test_split

In [205]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 0)

In [206]:
from sklearn.preprocessing import StandardScaler

In [207]:
sc=StandardScaler()
X_Train = sc.fit_transform(X_train)
X_Test = sc.transform(X_test)

In [208]:
#from sklearn.linear_model import LinearRegression
#lin = LinearRegression()
#lin.fit(X_train, y_train)

In [209]:
from sklearn.ensemble import RandomForestRegressor
lin = RandomForestRegressor(n_estimators=200, max_features=None)
lin.fit(X_train, y_train)

RandomForestRegressor(max_features=None, n_estimators=200)

In [210]:
def custom_accuracy(y_test, y_pred, thresold):
    right = 0
    l = len(y_pred)
    for i in range(0,l):
        if(abs(y_pred[i]-y_test[i]) <= thresold):
            right +=1
    return ((right/l)*100)

In [211]:
y_pred = lin.predict(X_test)
score = lin.score(X_test, y_test)*100
print("R-squared value: ",score)
print("Custom accuracy: ",custom_accuracy(y_test,y_pred,20))

R-squared value:  -24.798796099592725
Custom accuracy:  100.0


In [212]:
l =[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6,
       2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7,
       4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 5.1, 5.2, 5.3, 5.4, 5.5, 5.6, 2.7,
       1.7, 5.7, 5.8, 1.8, 4.7, 3.8, 0.8, 2.8, 1.9, 5.9, 4.8, 0.9, 3.9,
       4.9, 2.9]

In [213]:
input_test = pd.read_csv('input_test_data.csv')

In [214]:
input_test

,venue,innings,batting_team,bowling_team,batsmen,Unnamed: 5,Unnamed: 6,bowlers,Unnamed: 8,Unnamed: 9
0,Wankhede Stadium,1,Kolkata Knight Riders,Rajasthan Royals,N Rana,Shubman Gill,R A Tripathi,JD Unadkat,YBK Jaiswal,CH Morris


In [215]:
input_test['venue'] = input_test['venue'].replace(venue['avg_score'])

In [216]:
input_test['batting_team'] = input_test['batting_team'].replace(batting_team['avg_score'])

In [217]:
input_test['bowling_team'] = input_test['bowling_team'].replace(bowling_team['avg_score_conceded'])

In [218]:
input_test['batsmen'] = input_test['batsmen'].replace(batsman['strike_rate'])

In [219]:
input_test['Unnamed: 5'] = input_test['Unnamed: 5'].replace(batsman['strike_rate'])

In [220]:
input_test['Unnamed: 6'] = input_test['Unnamed: 6'].replace(batsman['strike_rate'])

In [221]:
input_test['bowlers'] = input_test['bowlers'].replace(bowler['economy'])

In [222]:
input_test['Unnamed: 8'] = input_test['Unnamed: 8'].replace(bowler['economy'])

In [223]:
input_test['Unnamed: 9'] = input_test['Unnamed: 9'].replace(bowler['economy'])

In [224]:
new_prediction=0
for i in l:
    new_prediction+= lin.predict(sc.transform(np.array(input_test[['venue', 'innings', 'batting_team', 'bowling_team', 'batsmen','Unnamed: 5','bowlers']])))
print("Prediction score: ",new_prediction)

Prediction score:  [14.97364286]
